In [4]:
## 실제 활용해보며 익히는 mysql (중급) 문법 

create table ranking(
    num int auto_increment not null primary key,
    main_category varchar(50) not null,
    sub_category varchar(50) not null,
    item_ranking tinyint unsigned not null,
    item_code varchar(20) not null,
    foreign key (item_code) references items(item_code)
);

SyntaxError: invalid syntax (<ipython-input-4-30fdf32cd950>, line 3)

In [5]:
create table items(
    item_code varchar(20) not null primary key,
    title varchar(200) not null,
    ori_price int not null,
    dis_price int not null,
    discount_percent int not null,
    provider varchar(100)
);

SyntaxError: invalid syntax (<ipython-input-5-82409d29c863>, line 1)

In [6]:
create schema gsshop default charset=utf8 collate=utf8_bin;
create table tablename(field definitions) default charset=utf8 collate=utf8_bin;

SyntaxError: invalid syntax (<ipython-input-6-0998f1dfb95f>, line 1)

In [7]:
import pymysql
host_name = "mariadb"
host_port = 3306
username = 'root'
password = '1234'
database_name = 'gsshop'

In [8]:
db = pymysql.connect (
    host=host_name,
    port=host_port,
    user=username,
    passwd=password,
    db=database_name,
    charset='utf8'
)

cursor = db.cursor()

sql = """
create table items(
    item_code varchar(20) not null primary key,
    title varchar(200) not null,
    ori_price int not null,
    dis_price int not null,
    discount_percent int not null,
    provider varchar(100)
);
"""
cursor.execute(sql)

sql = """
create table ranking(
    num int auto_increment not null primary key,
    main_category varchar(50) not null,
    sub_category varchar(50) not null,
    item_ranking tinyint unsigned not null,
    item_code varchar(20) not null,
    foreign key (item_code) references items(item_code)
);
"""
cursor.execute(sql)

db.commit()
db.close()

In [71]:
## 크롤링 GSSHOP
import requests as req
from bs4 import BeautifulSoup as bs
import pymysql

db = pymysql.connect(host="mariadb", port=3306, user="root", passwd="1234", db="devdb", charset="utf8")
cursor = db.cursor()

res = req.get('http://corners.gmarket.co.kr/Bestsellers')
soup = bs(res.content, 'html.parser')

## main category, link 가져오기 
categories = soup.select('div.gbest-cate ul.by-group li a')
for cate in categories:
    get_category('http://corners.gmarket.co.kr/' + cate['href'], cate.get_text())

http://corners.gmarket.co.kr//Bestsellers ALL
{'category_name': 'ALL', 'sub_category_name': 'ALL', 'ranking': 1, 'title': '[썬업]매일 썬업 과일야채샐러드 그린 200ml 48팩/주스', 'ori_price': '25400', 'dis_price': '19000', 'discount_percent': '25', 'item_code': '1558925634&ver', 'provider': ''}
{'category_name': 'ALL', 'sub_category_name': 'ALL', 'ranking': 2, 'title': '[한상궁김치]100% 국산 겉절이 2kg /출고당일제조', 'ori_price': '10900', 'dis_price': '10900', 'discount_percent': 0, 'item_code': '1798110680&ver', 'provider': ''}
{'category_name': 'ALL', 'sub_category_name': 'ALL', 'ranking': 3, 'title': '[라움에디션](시흥점)버켄스탁 [아리조나 에바 지제 블랙 스톤 모카] 6가지', 'ori_price': '22050', 'dis_price': '22050', 'discount_percent': 0, 'item_code': '2088402556&ver', 'provider': ''}
{'category_name': 'ALL', 'sub_category_name': 'ALL', 'ranking': 4, 'title': '스테비아 스윗 토마토 1kg+1kg (총 2팩)/추가40%할인', 'ori_price': '21400', 'dis_price': '21400', 'discount_percent': 0, 'item_code': '2079224426&ver', 'provider': ''}
{'category_name': 'ALL', 'sub_catego

KeyboardInterrupt: 

In [70]:
## sub category 정보 가져오기

def get_category(category_link, category_name):
    print(category_link, category_name)
    res = req.get(category_link)
    soup = bs(res.content, 'html.parser')
    
    ## 전체 랭킹 추출 ALL 이라고 임의로 이름을 생성
    get_items(soup, category_name, 'ALL')
    
    sub_categories = soup.select('div.navi.group ul li > a')
    for sub_category in categories:
        res = req.get('http://corners.gmarket.co.kr/' + sub_category['href'])
        soup = bs(res.content, 'html.parser')
        get_items(soup, category_name, sub_category.get_text())
        #print(category_link, category_name, sub.get_text(), 'http://corners.gmarket.co.kr/' + sub['href'])

In [69]:
## main / sub category + 상품 정보 + 상품 코드 + 판매자(제공자) 크롤링
def get_items(html, category_name, sub_category_name):
    items_result_list = list()
    best_item = html.select('div.best-list')
    
    for index, item in enumerate(best_item[1].select('li')):
        data_dict = dict()
        
        ranking = index + 1
        title = item.select_one('a.itemname')
        ori_price = item.select_one('div.o-price')
        dis_price = item.select_one('div.s-price strong span')
        discount_percent = item.select_one('div.s-price em')
        
        ## 태그 혹은 값이 없을 경우 .get_text() 제외 
        if ori_price == None or ori_price.get_text() == '':
            ori_price = dis_price
        
        if dis_price == None:
            ori_price, dis_price = 0, 0
        else:
            ## 문자열을 숫자로 변경 
            ori_price = ori_price.get_text().replace(',', '').replace('원', '')
            dis_price = dis_price.get_text().replace(',', '').replace('원', '')
        
        if discount_percent == None or discount_percent.get_text() == '':
            discount_percent = 0
        else:
            discount_percent = discount_percent.get_text().replace('%', '')
            
        ## 판매자 정보 가져오기
        product_link = item.select_one('div.thumb > a')
        ## 문자열을 = 기준으로 나누기
        item_code = product_link.attrs['href'].split('=')[1]
        
        res = req.get(product_link.attrs['href'])
        soup = bs(res.content, 'html.parser')
        provider = soup.select_one('div.item-topinfo_headline > p > a > strong')
        if provider == None:
            provider = ''
        else:
            provider = provider.get_text()
            
        data_dict['category_name'] = category_name
        data_dict['sub_category_name'] = sub_category_name
        data_dict['ranking'] = ranking
        data_dict['title'] = title.get_text()
        data_dict['ori_price'] = ori_price
        data_dict['dis_price'] = dis_price
        data_dict['discount_percent'] = discount_percent
        data_dict['item_code'] = item_code
        data_dict['provider'] = provider
        
        save(data_dict)
        #print(category_name, sub_category_name, ranking, provider, item_code, title.get_text(), ori_price, dis_price, discount_percent)
    

In [ ]:
def save(item_info):

    sql = """SELECT COUNT(*) FROM items WHERE item_code = '""" + item_info['item_code'] + """'"""
    cursor.execute(sql)
    result = cursor.fetchone()
    if result[0] == 0:         
        sql = """
        INSERT INTO items VALUES('""" + item_info['item_code'] + """', 
        '""" + item_info['title'] + """', 
        """ + str(item_info['ori_price']) + """, 
        """ + str(item_info['dis_price']) + """, 
        """ + str(item_info['discount_percent']) + """,
        '""" + item_info['provider'] + """'
        );
        """
        cursor.execute(sql)
        
    sql = """
        INSERT INTO ranking (main_category, sub_category, item_ranking, item_code) VALUES('""" + item_info['category_name'] + """', 
        '""" + item_info['sub_category_name'] + """', 
        '""" + str(item_info['ranking']) + """', 
        '""" + item_info['item_code'] + """'
        );
        """
    cursor.execute(sql)